In [3]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.1 MB/s eta 0:00:00


In [4]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
import numpy as np
from faker import Faker


In [5]:
client  = QdrantClient(host="localhost", port=6333)
client

In [6]:
my_collection = "first_collection"
client.create_collection(
    collection_name=my_collection,
    vectors_config=models.VectorParams(size=100, distance=models.Distance.COSINE)
)

True

In [7]:
data = np.random.uniform(low=-1.0, high=1.0, size=(1000,100))
index = list(range(1000))
print(data.shape)

(1000, 100)


In [8]:
data[:2, :5]

array([[-0.8151875 ,  0.26946907,  0.60930361,  0.29692612,  0.93258552],
       [-0.52344853,  0.4887113 ,  0.95136738, -0.78396036,  0.02563133]])

In [9]:
client.upsert(
    collection_name=my_collection,
    points=models.Batch(
        ids=index,
        vectors=data.tolist()
    )
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [10]:
client.retrieve(
    collection_name=my_collection,
    ids=[10,14,500]
)

[Record(id=10, payload={}, vector=None, shard_key=None, order_value=None),
 Record(id=14, payload={}, vector=None, shard_key=None, order_value=None),
 Record(id=500, payload={}, vector=None, shard_key=None, order_value=None)]

In [11]:
fake_something = Faker()
fake_something.name(), fake_something.address()

('James Moore', '41840 Roberts Point Apt. 429\nLake Ricky, VA 38863')

In [12]:
payload = []
for i in range(1000):
    payload.append(
        {
            "artist": fake_something.name(),
            "song": " ".join(fake_something.words()),
            "url_song": fake_something.url(),
            "year": fake_something.year(),
            "country": fake_something.country(),
        }
    )

payload[:2]

[{'artist': 'Joshua Reynolds',
  'song': 'continue participant ground',
  'url_song': 'http://burton.com/',
  'year': '2013',
  'country': "Cote d'Ivoire"},
 {'artist': 'Sherry Brooks',
  'song': 'concern receive someone',
  'url_song': 'http://www.tapia.com/',
  'year': '1980',
  'country': 'French Polynesia'}]

In [13]:
client.upsert(
    collection_name=my_collection,
    points=models.Batch(
        ids=index,
        vectors=data.tolist(),
        payloads=payload
    )
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

## Semantic Search

In [14]:
random_vec = np.random.uniform(low=-1.0, high=1.0, size=(100)).tolist()

In [15]:
client.search(
    collection_name=my_collection,
    query_vector=random_vec,
    limit=5
)

[ScoredPoint(id=925, version=1, score=0.29106066, payload={'artist': 'Matthew Hall', 'song': 'always yeah modern', 'url_song': 'http://krueger-lewis.biz/', 'year': '1984', 'country': 'Anguilla'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=556, version=1, score=0.2597546, payload={'artist': 'Nicholas Hall', 'song': 'think painting tough', 'url_song': 'https://www.wagner.info/', 'year': '2012', 'country': 'Montenegro'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=822, version=1, score=0.25470805, payload={'artist': 'Darlene Dalton', 'song': 'machine mean difficult', 'url_song': 'http://www.lamb.com/', 'year': '1975', 'country': 'Greece'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=700, version=1, score=0.25292808, payload={'artist': 'Christina Bruce', 'song': 'off himself sit', 'url_song': 'http://www.stephenson.com/', 'year': '2014', 'country': 'Ireland'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=965, v

In [22]:
aussie_songs = models.Filter(
    must= [
        models.FieldCondition(
            key="country", match=models.MatchValue(value="Montenegro")
        )
    ]
)

In [25]:
client.search(
    collection_name=my_collection,
    query_vector=random_vec,
    query_filter=aussie_songs,
    limit=5
)

[ScoredPoint(id=556, version=1, score=0.2597546, payload={'artist': 'Nicholas Hall', 'song': 'think painting tough', 'url_song': 'https://www.wagner.info/', 'year': '2012', 'country': 'Montenegro'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=44, version=1, score=0.21341681, payload={'artist': 'Douglas Cole', 'song': 'student example task', 'url_song': 'http://foster.biz/', 'year': '1998', 'country': 'Montenegro'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=387, version=1, score=0.053265657, payload={'artist': 'Cathy Williams', 'song': 'follow professor whom', 'url_song': 'http://kim.com/', 'year': '2009', 'country': 'Montenegro'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=819, version=1, score=0.045626536, payload={'artist': 'Kenneth Clarke MD', 'song': 'glass view ten', 'url_song': 'http://nelson-hernandez.com/', 'year': '2023', 'country': 'Montenegro'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=539,